# Daily Average Glucose

This notebook generates daily average glucose chart from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service:

- A line plot for daily mean
- +/- 1 standard deviation shaded area, indicating daily variability
- Target zone indicator

Both charts include the median line.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, timedelta

# Reporting date range
start = date(2025, 8, 4)
end = start + timedelta(days=11)

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = []

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Set the Y-axis limits
y_min = 3
y_max = 15

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Remove any excluded ranges
df = remove_records_for_date_ranges(df, exclusions)

# Preview the data
df.head()

In [ ]:
# Rename the date column to timestamp and generate a date-only column
df.rename(columns={ "date": "timestamp" }, inplace=True)
df["date"] = df["timestamp"].dt.date

# Group by date and calculate mean and standard deviation
daily_stats = df.groupby("date")["level"].agg(["mean", "std"]).reset_index()

# Preview the data
daily_stats.head()

In [ ]:
# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("glucose_daily_mean", { "Daily Mean": daily_stats })

In [ ]:
import matplotlib.pyplot as plt

# Plot setup
plt.figure(figsize=(12, 6))

# Plot daily mean glucose
plt.plot(daily_stats["date"], daily_stats["mean"], label="Daily Mean Glucose (mmol/L)", marker="o")

# Plot ±1 standard deviation as a shaded area
plt.fill_between(
    daily_stats["date"],
    daily_stats["mean"] - daily_stats["std"],
    daily_stats["mean"] + daily_stats["std"],
    color="blue",
    alpha=0.2,
    label="±1 SD"
)

# Add target range band: 3.9–10.0 mmol/L
plt.axhspan(3.9, 10.0, color="green", alpha=0.1, label="Target Range (3.9–10.0 mmol/L)")

# Formatting
plt.title(f"Daily Average Glucose : {start.strftime('%d-%b-%Y')} to {end.strftime('%d-%b-%Y')}")
plt.xlabel("Date")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.xticks(rotation=45)
plt.grid(True)

# Legend below chart
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.25), ncol=2)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_daily_mean", None, chart_export_format)

# Show the plot
plt.show()
